In [1]:
%reload_ext autoreload
%autoreload 2

In [29]:
import param as pm
import pandas as pd
import panel as pn
pn.extension('echarts')

class Distribution(pm.Parameterized):
    """
    Shawn, LTF, Feb 4th 2022
    This class represents a distribution. A distribution is a vector that sums 
    to one, in mathematics, often refered to as a 'stochastic vector'. A 
    stochastic vector represents a probability distribution that sums to one. 
    
    The vector maintains a sum of one with the following formula:
    
    When a_j is adjusted:
    
    for all allocations a_i | a_i != a_j:
        a_i := a_i * (1 - a_j) / sum(a_i | a_i !=a_j)
    
    
    Usage:
        First add any distribution elements as parameterized numbers to this class. 
        Additionally add the name of the distribution elements to the allocations 
        list attribute on this class. 
        
        The `_updating` attribute is used as an internal mechanism to prevent recursive
        updating of the parameters since they update themselves depending on themselves.
        
    This class could be more general by taking an allocation_list as an init param and 
    dynamically creating the allocation parameters. But dynamically adding and removing
    parameters to an object seems messy. I think inheritance is actually better. 
    """
    founders = pm.Number(0.1, bounds=(0,1), step=0.01)
    dao_rewards = pm.Number(0.5, bounds=(0,1), step=0.01)
    lbp_pool = pm.Number(0.4, bounds=(0,1), step=0.01)
    allocations = pm.List(['founders', 'dao_rewards', 'lbp_pool'], precedence=-1, constant=True)
    _updating = pm.Boolean(False, precedence=-1)
    _smallest_value = pm.Number(0.001, precedence=-1)

    def __init__(self, **params):
        super().__init__(**params)
        self.param.watch(self.normalize, self.allocations)
        self.normalize(event=self.param[self.allocations[0]])
        
    def normalize(self, event):
        """
        Sentinal design pattern inspired from here: 
        https://discourse.holoviz.org/t/updating-selected-value-in-widget-after-changing-param-selector-objects/1806/2
        """
        
        # Sentinal to prevent recursive updates
        if not self._updating:
            self._updating = True
            
            # List of all allocations except the one being updated
            normalize_allocations = list(self.allocations)
            normalize_allocations.remove(event.name)
            
            denominator = sum([getattr(self, a) for a in normalize_allocations])
            numerator = (1 - getattr(self, event.name))
            
            # Normal case
            if (denominator != 0):
                normalize_factor = numerator / denominator
                for a in normalize_allocations:
                    setattr(self, a, getattr(self, a) * normalize_factor)
                    
            # Exceptional case where denominator is 0 and the selected allocation has moved.
            elif numerator != 1:
                for a in normalize_allocations:
                    setattr(self, a, (1-getattr(self, event.name))/len(self.allocations))
                    
            # Exceptional case where everything arrives at 0
            if sum([getattr(self, a) for a in self.allocations]) == 0:
                for a in normalize_allocations:
                    setattr(self, a, 1/(len(self.allocations)-1))
            # Reset Sentinal
            self._updating = False
            
    def table(self):
        table = pd.DataFrame(self.param.get_param_values())
        table.columns = ['Elements', 'Allocations']
        return table.set_index('Elements').loc[self.allocations]
    
    def table_with_total(self):
        table = self.table()
        table.loc['total'] = table['Allocations'].sum()
        return table

d = Distribution()

pn.Row(d, d.table)

Row
    [0] Column(margin=5, name='Distribution', width=300)
        [0] StaticText(value='<b>Distribution</b>')
        [1] FloatSlider(name='Founders', step=0.01, value=0.1)
        [2] FloatSlider(name='Dao rewards', step=0.01, value=0.5)
        [3] FloatSlider(name='Lbp pool', step=0.01, value=0.4)
    [1] ParamMethod(method)

In [3]:
d.founders

0.1

In [4]:
class LTFDistribution(Distribution):
    racoon_fund = pm.Number(0.1, bounds=(0,1), step=0.01)

ltf_dist = LTFDistribution(allocations=['founders', 'dao_rewards', 'lbp_pool', 'racoon_fund'])

pn.Row(ltf_dist, ltf_dist.table_with_total)

Row
    [0] Column(margin=5, name='LTFDistribution', width=300)
        [0] StaticText(value='<b>LTFDistribution</b>')
        [1] FloatSlider(name='Founders', step=0.01, value=0.1)
        [2] FloatSlider(name='Dao rewards', step=0.01, value=0.45)
        [3] FloatSlider(name='Lbp pool', step=0.01, value=0.36000000000000004)
        [4] FloatSlider(name='Racoon fund', step=0.01, value=0.09000000000000001)
    [1] ParamMethod(method)

Founder Distribution

In [30]:
from parameters.fixtures import roles as role_parameters

class Role(pm.Parameterized):
    base_wage = pm.Number(2000, bounds=(0,10000), step=2000)
    description = pm.String("Add a description")
    
    def __init__(self, name: str, **params):
        super().__init__(name=name, **params)

roles = [Role(**r) for r in role_parameters]
pn.Row(*roles)

Row
    [0] Column(margin=5, name='CEO', width=300)
        [0] StaticText(value='<b>CEO</b>')
        [1] FloatSlider(end=10000, name='Base wage', step=2000, value=2000)
        [2] TextInput(name='Description', value='Highest level d...)
    [1] Column(margin=5, name='CFO', width=300)
        [0] StaticText(value='<b>CFO</b>')
        [1] FloatSlider(end=10000, name='Base wage', step=2000, value=2000)
        [2] TextInput(name='Description', value='Bottoms-lines guy.')
    [2] Column(margin=5, name='CTO', width=300)
        [0] StaticText(value='<b>CTO</b>')
        [1] FloatSlider(end=10000, name='Base wage', step=2000, value=2000)
        [2] TextInput(name='Description', value='Add a description')
    [3] Column(margin=5, name='Admin', width=300)
        [0] StaticText(value='<b>Admin</b>')
        [1] FloatSlider(end=10000, name='Base wage', step=2000, value=1600)
        [2] TextInput(name='Description', value='Add a description')
    [4] Column(margin=5, name='CMO', width=300)
        [0] StaticText(value='<b>CMO</b>')
        [1] FloatSlider(end=10000, name='Base wage', step=2000, value=2000)
        [2] TextInput(name='Description', value='Add a description')
    [5] Column(margin=5, name='Token Engineer 1', width=300)
        [0] StaticText(value='<b>Token Engineer 1</b>')
        [1] FloatSlider(end=10000, name='Base wage', step=2000, value=2000)
        [2] TextInput(name='Description', value='Add a description')
    [6] Column(margin=5, name='Token Engineer 2', width=300)
        [0] StaticText(value='<b>Token Engineer 2</b>')
        [1] FloatSlider(end=10000, name='Base wage', step=2000, value=3000)
        [2] TextInput(name='Description', value='Add a description')
    [7] Column(margin=5, name='Racoon', width=300)
        [0] StaticText(value='<b>Racoon</b>')
        [1] FloatSlider(end=10000, name='Base wage', step=2000, value=3000)
        [2] TextInput(name='Description', value='Add a description')

In [6]:
class Profile(pm.Parameterized):
    email = pm.String()
    discord = pm.String()
    github = pm.String() 
    twitter = pm.String()
    ethereum = pm.String()
    
p = Profile()
pn.Pane(p)

Param(Profile)

In [7]:
import datetime as dt
import numpy as np

class Founder(pm.Parameterized):
    first_name = pm.String()
    last_name  = pm.String()
    NDA        = pm.Boolean(True)
    onboarded  = pm.Boolean(True)
    active     = pm.Boolean(True)
    contract   = pm.Boolean(True)
    engagement = pm.ObjectSelector(default='Full Time', objects=['Full Time', 'Part Time'])
    authorization_level = pm.Integer(1, bounds=(1,5))
    credentials_level = pm.Integer(1, bounds=(1,5))
    conviction_level  = pm.Integer(1, bounds=(1,5))
    role = pm.ObjectSelector(default=roles[0], objects=roles)   
    date_joined =  pm.Date(dt.date(2018, 8, 18), bounds=(dt.date(2018, 8, 18), None))
    profile = pm.ObjectSelector(default=p, objects=[p]) 
    
    def clearance(self):
        return self.NDA * self.onboarded * self.active * self.contract
    
    def months_since_joining(self):
        d1 = dt.date.today()
        d2 = self.date_joined
        months_since_joining = (d1.year - d2.year) * 12 + d1.month - d2.month
        return months_since_joining
    
    def time_bonus(self):
        months_since_joining = self.months_since_joining()
        return round(np.sqrt(months_since_joining+1)/2, 2)
        
    def rewards_multiplier(self):
        return round(np.sqrt(self.conviction_level * self.credentials_level * self.authorization_level)/2, 2)
    
    def base_wage(self):
        return (1 if self.engagement=='Full Time' else 0.5) * self.role.base_wage
    
    def reward_schedule(self):
        rewards = self.clearance() * self.base_wage() * (self.time_bonus() + self.rewards_multiplier()) 
        return round(rewards)
    
    def clearance_gauge(self):
        return pn.indicators.Gauge(
            name='Clearance', value=self.clearance(), bounds=(0, 1), format='{value} ✅',
            colors=[(0.2, 'green'), (0.8, 'gold'), (1, 'red')], annulus_width=5, custom_opts={'radius':120})
    
    def time_bonus_gauge(self):
        return pn.indicators.Gauge(
            name='Time Bonus', value=self.time_bonus(), bounds=(0, 100), format='{value} ✅',
            colors=[(0.2, 'green'), (0.8, 'gold'), (1, 'red')], annulus_width=5, custom_opts={'radius':120})
    
    def rewards_multiplier_gauge(self):
        return pn.indicators.Gauge(
            name='Rewards Multiplier', value=self.rewards_multiplier(), bounds=(1, 10), format='{value} ✅',
            colors=[(0.2, 'green'), (0.8, 'gold'), (1, 'red')], annulus_width=5, custom_opts={'radius':120})
    
    @pn.depends('role.base_wage')
    def base_wage_gauge(self):
        return pn.indicators.Gauge(
            name='Base Wage', value=self.base_wage(), bounds=(0, 8000), format='{value} ✅',
            colors=[(0.2, 'green'), (0.8, 'gold'), (1, 'red')], annulus_width=5, custom_opts={'radius':120})
    
    @pn.depends('role.base_wage')
    def rewards_gauge(self):
        return pn.indicators.Gauge(
            name='Rewards', value=self.reward_schedule(), bounds=(0, 8000), format='{value} 💸',
            colors=[(0.2, 'green'), (0.8, 'gold'), (1, 'red')])
    
    
#     @pn.depends('role.base_wage')
    def view_reward_gauge(self):
        
        g1 = self.clearance_gauge

        g2 = self.time_bonus_gauge
        
        g3 = self.rewards_multiplier_gauge
        
        g4 = self.base_wage_gauge
        
        g5 = self.rewards_gauge
        
        return pn.Column(pn.Row(g1, g2, g3), pn.Row(g4, g5))
        
    
    def view(self):
        founder = pn.Param(self, widgets={
            'date_joined': pn.widgets.DatePicker(name='Date Joined', value=self.date_joined)
        })
        return founder

In [8]:
from parameters.fixtures import founders as founder_parameters

DAO Rewards

In [9]:
founders = [Founder(**f,role=pm.named_objs(roles)[f['role_name']]) for f in founder_parameters]

In [27]:
pn.Row(*[pn.Row(f.view(), f.view_reward_gauge) for f in founders])

Row
    [0] Row
        [0] Param(Founder, widgets={'date_joined': DatePicker...})
        [1] ParamMethod(method)
    [1] Row
        [0] Param(Founder, widgets={'date_joined': DatePicker...})
        [1] ParamMethod(method)
    [2] Row
        [0] Param(Founder, widgets={'date_joined': DatePicker...})
        [1] ParamMethod(method)
    [3] Row
        [0] Param(Founder, widgets={'date_joined': DatePicker...})
        [1] ParamMethod(method)
    [4] Row
        [0] Param(Founder, widgets={'date_joined': DatePicker...})
        [1] ParamMethod(method)
    [5] Row
        [0] Param(Founder, widgets={'date_joined': DatePicker...})
        [1] ParamMethod(method)
    [6] Row
        [0] Param(Founder, widgets={'date_joined': DatePicker...})
        [1] ParamMethod(method)
    [7] Row
        [0] Param(Founder, widgets={'date_joined': DatePicker...})
        [1] ParamMethod(method)
    [8] Row
        [0] Param(Founder, widgets={'date_joined': DatePicker...})
        [1] ParamMethod(method)

In [11]:
len(founders)

9

In [12]:
f = founders[0]

In [13]:
rewards = [f.reward_schedule() for f in founders]

In [14]:
{f.first_name+' '+f.last_name: f.reward_schedule() for f in founders}

{'Shawn Anderson': 17420,
 'Aidan Oconnel': 4008,
 'Mike De Melo': 12540,
 'Jake Cassani': 12440,
 'Liam Perison': 6960,
 'BK K': 10700,
 'Katie Knorr': 6180,
 'Andrew Penland': 6510,
 'Juugaahd S': 8288}

In [15]:
sum(rewards) * 2

170092

In [16]:
total = {}
for f in founders:
    reward_schedule = []
    for i in range(f.months_since_joining()):
        reward_schedule.append(f.reward_schedule())
        f.date_joined += dt.timedelta(days=30)
    total[f.first_name+' '+f.last_name] = reward_schedule

In [17]:
total

{'Shawn Anderson': [17420,
  17340,
  17280,
  17220,
  17140,
  17080,
  17000,
  16920,
  16860,
  16780,
  16700,
  16640,
  16560,
  16480,
  16400,
  16320,
  16240,
  16160,
  16080,
  16000,
  15920,
  15840,
  15740,
  15660,
  15560,
  15480,
  15380,
  15300,
  15200,
  15100,
  15000,
  14900,
  14800,
  14700,
  14580,
  14480,
  14360,
  14240,
  14120,
  14120,
  14000,
  13880,
  13740,
  13600,
  13460,
  13320,
  13160,
  13000,
  12820,
  12640,
  12440,
  12240,
  12000,
  11740],
 'Aidan Oconnel': [4008,
  3976,
  3944,
  3912,
  3880,
  3848,
  3816,
  3784,
  3752,
  3720,
  3680,
  3648,
  3608,
  3576,
  3536,
  3504,
  3464,
  3424,
  3384,
  3344,
  3304,
  3264,
  3216,
  3176,
  3128,
  3080,
  3032,
  2984,
  2936,
  2880,
  2824,
  2768,
  2712,
  2648,
  2584,
  2512,
  2440,
  2360,
  2280,
  2280,
  2184,
  2080],
 'Mike De Melo': [12540,
  12440,
  12320,
  12220,
  12100,
  11980,
  11860,
  11740,
  11620,
  11480,
  11340,
  11200,
  11060,
  10900,

In [18]:
sum_totals = {k:sum(v) for k,v in total.items()}
sum_totals

{'Shawn Anderson': 817140,
 'Aidan Oconnel': 134480,
 'Mike De Melo': 245020,
 'Jake Cassani': 232480,
 'Liam Perison': 123152,
 'BK K': 195940,
 'Katie Knorr': 100090,
 'Andrew Penland': 106030,
 'Juugaahd S': 131344}

In [19]:
sum(sum_totals.values()) * 2

4171352

In [20]:
2382332 / 9

264703.55555555556

In [21]:
264703.55555555556 / 2000000

0.1323517777777778

In [22]:
sum(rewards) / len(rewards)

9449.555555555555

In [23]:
sum(rewards) * 2

170092

In [24]:
sum(rewards) * 2 * 12

2041104

In [25]:
 1822320 / 9

202480.0

In [26]:
202480.0 / 2000000

0.10124

LBP Raise Amount 20-40 million

Here, we integrate an LBP model to calculate the number of tokens sold, and the amount of funds raised. 

Racoon Fund